<a href="https://colab.research.google.com/github/AbhiMalappa/NLP/blob/main/helpdesk_BERT_predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from torch import nn
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import pipeline
import torch

! pip install -q streamlit
import streamlit as st

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 125.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 125.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 7.6 MB/s eta 0:00:00


In [2]:
! pip install -q streamlit
import streamlit as st

### Function to get predictions

*   Model - Transfer Learning using BERT model, finetuned on help desk data, trained and saved previously. Call saved check point.  
*   Tokenize - new data using Autotokenized from pretrained on BERT base.
*   Output is logits. Convert to probabilities using Pytorch NN softmax func.
*   Result is a dict. Handle it as per downstream requirement.

*   Creating a pipeline on the saved model is the easiest way. But pipeline class returns only the top probability classification. If we want all classes and probabilities for all classes, use above method.




In [3]:
# funct to get predictions
#model_ckpt = "drive/MyDrive/Colab Notebooks/NLP/trained_model/Finetune_BERT_for_helpdesk_issue_classification"
model_ckpt = "drive/MyDrive/Colab Notebooks/NLP/trained_model/V2_Finetune_BERT_for_helpdesk_issue_classification_7_class"

# prediction for all labels
def get_pred(text):
  device = torch.device("cuda")

  #read model
  model = AutoModelForSequenceClassification.from_pretrained(model_ckpt).to(device)

  # from model config get labels names
  labels = model.config.id2label.values()
  labels = list(labels)

  #tokenize input
  tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
  input_encoded = tokenizer(text, return_tensors='pt').to(device)

  # get model pred on tokenized input
  with torch.no_grad():
    outputs = model(**input_encoded) # output is a torch tensor with logit values on gpu

  #output logits
  logits = outputs.logits[0] # get logit values
  #output probs
  probabilities = nn.functional.softmax(logits, dim=-1) # convert logit to prob

  # return a dic with labels and probs
  probabilities = probabilities.cpu().detach().numpy() # convert to numpy on cpu
  result = {labels[i]: probabilities[i] for i in range(len(labels))}

  return result, #probabilities

# prediction return top label
def pipeline_clf(text):
  device = torch.device("cuda")
  clf = pipeline('text-classification',model = model_ckpt)
  pipeline_clf_label = clf(text)[0]['label']
  pipeline_clf_score = clf(text)[0]['score']
  return pipeline_clf_label, pipeline_clf_score

# get sentiment
def sentiment(text):
  sentement_obj = pipeline('text-classification', model = "cardiffnlp/twitter-roberta-base-sentiment-latest")
  #sentement_obj = pipeline('sentiment-analysis')
  #emotions
  #sentement_obj = pipeline(task="text-classification", model="SamLowe/roberta-base-go_emotions", top_k=5)

  sentiment = sentement_obj(text)
  return sentiment

# pass custom input
def custom_input(text):
  pred_confidence = {}
  sentiment_val = sentiment(text)
  pred = get_pred(text)[0]
  classification_list = ['agent_unprofessional','long_holdtime','multiple_contacts_made','no_resolution','service_portal', 'Others']

  for keys in pred:
    if keys in classification_list and pred[keys] > 0.2:
      pred_confidence[keys] = pred[keys]

  if len(pred_confidence) == 0:
    pred_confidence['Others'] = 0.3

  if sentiment_val[0]['score'] > 0.6:
    pred_confidence[sentiment_val[0]['label'] ] = sentiment_val[0]['score']

  return pred_confidence, pred


### Predict on new data

In [ ]:
# connect to IDW SQL DB
import pyodbc

driver = 'ODBC Driver 17 for SQL Server'

conn = pyodbc.connect(
    driver = driver,
    server = 'bidw-prod-dw02.appl.kp.org',
    port = 1433,
    uid = 'svcCESMPROBCM',
    pwd = 'K@********')

SQLQuery=""" Select top 500 task_id, instance, category,metric, question,string_value,user,user_manager,u_segmentation,cmdb_ci,sys_updated_on, sys_created_by,
            loaded_date_id,instance_task_id_sys_created_on, instance_task_id_closed_at, instance_task_id_assigned_to ,[Assignment Group] as instance_task_id_assignemnt_group, contact_type"
            FROM SM_DW.dbo.vw_ASMT_METRIC_RESULT_ULM_CSATIncident WITH (NOLOCK)
            WHERE [SCALED_VALUE] = '-1' AND [STRING_VALUE] IS NOT NULL AND instance_task_id_sys_created_on > '2023-02-01'
        """
print(SQLQuery)

#crsr = conn.cursor()
t0=time.time()
#df = pd.read_sql(SQLQuery, conn)
print("Rows = ",df.shape[0])
t1=time.time()
print(f"Execution Time= {t1- t0}")

#Run this to close otherwise Art and Harjinder will be unhappy
#crsr.close()
#conn.close()

#df.head()

Add code to split int scentence and then predict

In [4]:
import pandas as pd
import numpy as np
df = pd.read_csv('drive/MyDrive/Colab Notebooks/training_text_calssifiaction.csv')
df = df.sample(3)

# call prediction
df['pred_label'] = df['string_value'].apply(lambda x: pipeline_clf(x)[0])
df['pred_label_score'] = df['string_value'].apply(lambda x: pipeline_clf(x)[1])
df['pred_label'] = np.where(df['pred_label_score'] < 0.6, 'Others', df['pred_label'])

df['agent_unprofessional_score'] = df['string_value'].apply(lambda x: get_pred(x)[0]['agent_unprofessional'])
df['long_holdtime_score'] = df['string_value'].apply(lambda x: get_pred(x)[0]['long_holdtime'])
df['multiple_contacts_made_score'] = df['string_value'].apply(lambda x: get_pred(x)[0]['multiple_contacts_made'])
df['no_resolution_score'] = df['string_value'].apply(lambda x: get_pred(x)[0]['no_resolution'])
df['service_portal_score'] = df['string_value'].apply(lambda x: get_pred(x)[0]['service_portal'])
df['pred_val_all_class'] = df['string_value'].apply(lambda x: get_pred(x))

df['sentiment_label'] = df['string_value'].apply(lambda x: sentiment(x)[0]['label'])
df['sentiment_score'] = df['string_value'].apply(lambda x: sentiment(x)[0]['score'])

df.head(2)

Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
Device set to use cuda:0
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Device set to use cuda:0
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected 

,string_value,Category,pred_label,pred_label_score,agent_unprofessional_score,long_holdtime_score,multiple_contacts_made_score,no_resolution_score,service_portal_score,pred_val_all_class,sentiment_label,sentiment_score
2959,waiting time on hold was too long.,long_holdtime,long_holdtime,0.986529,0.001707,0.986529,0.003443,0.002795,0.002472,"({'agent_unprofessional': 0.001707376, 'long_h...",negative,0.831679
2296,i had to wait a short while for my call to be ...,long_holdtime,long_holdtime,0.985957,0.001884,0.985957,0.004030,0.002735,0.002104,"({'agent_unprofessional': 0.0018835897, 'long_...",neutral,0.681006


### Predict on text input

In [72]:
text = [
    " I felt like the service rep was not patient enough to explain what the issue was and how i can prevent it from happening again. Probably having a bad day. I was not satisfied with this service. "
    ]

custom_input(text)[0]


Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


{'long_holdtime': 0.9840379, 'neutral': 0.7411251664161682}

In [63]:
get_pred(text)[0]

{'agent_unprofessional': 0.0017623379,
 'long_holdtime': 0.98534554,
 'multiple_contacts_made': 0.0036236856,
 'negative_feedback': 0.0018375481,
 'no_resolution': 0.003192431,
 'positive_feedback': 0.0016503453,
 'service_portal': 0.0025881093}

### Zero-Shot Classification

In [119]:
zero_short_pipe = pipeline(model="facebook/bart-large-mnli")

def zero_short(text):
  result = zero_short_pipe(text, candidate_labels=["unprofessional", "long wait", "not resolved", "positive", "negative", "multiple contacts", "lack knowledge"], )
  ab_dict = {}
  for x in range(len(result['labels'])):
    ab_dict[result['labels'][x]] = result['scores'][x]

  sentiment_val = sentiment(text)
  if sentiment_val[0]['label'] == 'positive':
    ab_dict['long wait'] = ab_dict['long wait'] * 0.1
    ab_dict['not resolved'] = ab_dict['not resolved'] * 0.1
    ab_dict['unprofessional'] = ab_dict['unprofessional'] * 0.1

  pred_conf = {}

  for keys in ab_dict:
    if ab_dict[keys] > 0.2:
      pred_conf[keys] = ab_dict[keys]
  ab_dict = pred_conf


  ab_dict.pop('positive', None)
  ab_dict.pop('negative', None)
  ab_dict["sentiment-" + sentiment_val[0]['label'] ] = sentiment_val[0]['score']
  ab_df = pd.DataFrame.from_dict(ab_dict, orient='index', columns=['Probability'])
  ab_df.reset_index(inplace=True)
  ab_df = ab_df.rename(columns={'index': 'Classification Tags'})

  return ab_df



Device set to use cuda:0


In [121]:
text = "agent was unhelpful and i had to wait long "
zero_short(text)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use cuda:0


,Classification Tags,Probability
0,long wait,0.407057
1,unprofessional,0.354719
2,sentiment-negative,0.926068


#### *Using Gradio for demo*

In [122]:
!pip install -q gradio
import gradio

iface = gr.Interface(
    fn=zero_short,
    inputs="text",
    outputs=gr.DataFrame(headers=["Tag", "Probability"]),
    title="Text Classification - Finetuned BERT Large Language Model",
    description = """


    IT helpdesk is involded in supporting variety of task involving simplest like password reset to troubleshooting complex issues. After a call with helpdesk agent, end user provided feedback through text and voice note. AI is used to analyze each review/feedback and score the performance of the helpdesk agent.

    Each review is analysed to find these pre-defined tags - 'Issue not resolved', 'Long Hold Time', 'Agent Unrpfessional', 'Agent Lacks Knowledge', 'Multiple contacts/Followup calls' and 'Sentiment'.

    \nExample 1:\n
    Input
    "Took about over a week to resolve the ticket. I was passed around to multiple departments and had talked to at least 8 ITs."
    Output
    multiple contacts	   score 0.674543

    \nExample 2:\n\n
    \nInput
    "Representative was very helpful and nice. I am very grateful for his help. "
    Output
    sentiment-positive	 score 0.985908

    \n
    Example 3:
    Input
    "The technician was not listening she was talking over me and would not let me finish what was the problem."
    Output
    unprofessional	     score 0.674543
    sentiment-negative	 score 0.892530

    """,

    article = """
                 Model - Bert base. 110M parameters Encoder Trasformer \n
                 Contact - Abhiraj Malappa (amalappa@bitsioinc.com)
              """
)

iface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7075bd2e6a36b744dd.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [159]:
%%writefile app.py
import streamlit as st

st.write("Hello World")


Overwriting app.py


In [160]:
!npm install localtunnel

⠙⠹⠸⠼⠴⠦
up to date, audited 23 packages in 861ms
⠦
⠦3 packages are looking for funding
⠦  run `npm fund` for details
⠦
2 moderate severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠦

In [161]:
!wget -q -O - ipv4.icanhazip.com
#!curl https://loca.lt/mytunnelpassword


34.34.68.109


In [162]:
#!streamlit run /content/app.py &>/content/logs.txt &
!streamlit run app.py &>/content/logs.txt &


In [163]:
!npx localtunnel --port 8501

⠙your url is: https://fluffy-trams-grab.loca.lt
^C


In [166]:
def greet(name):
  return "Hello " + name + "!"

greet("World")

'Hello World!'

In [11]:
!pip install -q gradio
import gradio

gradio.Interface(zero_short, "text", "text", title="text classifiaction").launch(share=True)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.4/321.4 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 126.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.1 MB/s eta 0:00:00
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://98b87deb4d7c0329f9.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
